In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from brisc.manuscript_analysis import distance_between_cells as dist_cells


from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams["pdf.fonttype"] = 42  # for pdfs

import flexiznam as flz

In [ ]:
processed_path = Path(
    "/nemo/project/proj-znamenp-barseq/processed/becalia_rabies_barseq/BRAC8498.3e/"
)
rabies_cell_properties = dist_cells.load_presynaptic_distances(
    processed_path,
)

relative_presyn_coords, distances = dist_cells.determine_presynaptic_distances(
    rabies_cell_properties
)

all_shuffled_distances, all_starter_coords = (
    dist_cells.create_barcode_shuffled_nulls_parallel(
        rabies_cell_properties, N_iter=1000
    )
)

In [ ]:
# Plot Fig.1
fontsize_dict = {"title": 7, "label": 7, "tick": 5, "legend": 5}
pad_dict = {"label": 1, "tick": 1, "legend": 5}
line_width = 0.9
line_alpha = 1

cm = 1 / 2.54
fig = plt.figure(figsize=(18 * cm, 18 * cm), dpi=350)
data_path = Path(
    "/nemo/lab/znamenskiyp/home/shared/projects/barcode_diversity_analysis/collapsed_barcodes/"
)
save_path = Path(
    "/nemo/lab/znamenskiyp/home/shared/projects/barcode_diversity_analysis/figures/"
)
save_fig = False
figname = "fig1_plasmid_barcoding_schema_library"

# 1 Presynaptic distance shared barcode
# 2 Presynaptic distance shuffled barcode

# Coord format: [left, bottom, width, height]
ax_locations = [
    [0.05, 0.67, 0.25, 0.25],  # Row 1, Col 1
    [0.46, 0.67, 0.25, 0.25],  # Row 1, Col 2
    [0.05, 0.49, 0.25, 0.25],  # Row 2, Col 1
]

ax_presyn_shared = fig.add_axes(ax_locations[0])

dist_cells.plot_AP_ML_relative_coords(
    relative_presyn_coords,
    ax=ax_presyn_shared,
    lims={"x": (-1000, 1000), "z": (-3000, 3000)},
    s=1,
    alpha=0.1,
    color="black",
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
)

ax_presyn_shuffled = fig.add_axes(ax_locations[2])

dist_cells.plot_AP_ML_relative_coords(
    all_shuffled_distances[0],
    ax=ax_presyn_shuffled,
    lims={"x": (-1000, 1000), "z": (-3000, 3000)},
    s=1,
    alpha=0.1,
    color="black",
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
)

ax_dist_histo = fig.add_axes(ax_locations[1])
dist_cells.plot_3d_distance_histo(
    distances,
    all_shuffled_distances,
    ax=ax_dist_histo,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    bins=100,
    max_dist=5,
    linewidth=0.9,
)

if save_fig:
    pdf_path = f"{save_path}/figures/{figname}.pdf"
    png_path = f"{save_path}/figures/{figname}.png"
    fig.savefig(pdf_path, format="pdf", transparent=True, bbox_inches="tight")
    fig.savefig(png_path, format="png", transparent=True, bbox_inches="tight")
    print(f"Saved PDF figure to {pdf_path}")
    print(f"Saved PNG figure to {png_path}")